
(tutorials-hubbard-parallel)=

# Parallelizing the computation of Hubbard parameters

In this tutorial you will learn how to parallelize the computation of the Hubbard parameters using the {py:class}`~aiida_hubbard.workflows.hp.main.HpWorkChain`.

We can divide this goal in two phases:

* __Parallelize over independent atoms__: parallelize the ``hp.x`` calculation with multiple sub-``hp.x`` running single atoms.
* __Parallelize over independent q points__: parallelize each atom sub-``hp.x`` with other sub-``hp.x`` running single q points.

As we learnt from the [previous tutorial](./1_computing_hubbard.ipynb), first we need to compute the ground-state with a ``pw.x`` calculation.

Let's get started!

In [ ]:
from local_module import load_temp_profile

# If you download this file, you can run it with your own profile.
# Put these lines instead:
# from aiida import load_profile
# load_profile()
data = load_temp_profile(
    name="hubbard-parallel-tutorial",
    add_computer=True,
    add_pw_code=True,
    add_hp_code=True,
    add_sssp=True,
    add_structure_licoo=True,
)

In [ ]:
from aiida.engine import run_get_node
from aiida.orm import KpointsData
from aiida_quantumespresso.workflows.pw.base import PwBaseWorkChain
from aiida_quantumespresso.common.types import ElectronicType
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1,1,1])

builder = PwBaseWorkChain.get_builder_from_protocol(
    code=data.pw_code, # modify here if you downloaded the notebook
    structure=data.structure, # modify here if you downloaded the notebook
    protocol="fast",
    electronic_type=ElectronicType.INSULATOR,
    overrides={"kpoints":kpoints, "clean_workdir":False}
)
results, pw_node = run_get_node(builder)

## Parallelize over atoms

To parallelize over atoms, we need a _new_ workchain which is dedicated to this purpose: the {py:class}`~aiida_hubbard.workflows.hp.main.HpWorkChain`. This workchain is able to parallelize both over atoms and over q points.

Let's see first the atom parallelization. As usual, we need to get the `builder` and fill the inputs.
Specifying the input `parallelize_atoms` as `True` in `HpWorkChain`, each _independent atom_ will be run as a separate `HpBaseWorkChain`.

In [ ]:
from aiida_hubbard.workflows.hp.main import HpWorkChain

builder = HpWorkChain.get_builder_from_protocol(
    code=data.hp_code,
    protocol="fast",
    parent_scf_folder=pw_node.outputs.remote_folder,
    overrides={
        "parallelize_atoms":True, 
        "parallelize_qpoints":False, 
        "hp":{"hubbard_structure":data.structure},
        "qpoints_distance": 100.0, # to get few q points
        }
)

results, hp_node = run_get_node(builder)
results

Let's have a look at the workflow:

In [ ]:
%verdi process status {hp_node.pk}

The following just happened:
- A grid of q points is generated automatically using the distance (between points) in $\r{A}^{-1}$ we gave in input (of 100 $\r{A}^{-1}$ to have very sparse - it is just a tutorial!).
- The `HpParallelizeAtomsWorkChain` is called.
- This work chain calls first a `HpBaseWorkChain` to get the independent atoms to perturb.
- **Three** `HpBaseWorkChain` are submitted __simultaneously__, one for cobalt, and two for the two oxygen sites.
- The response matrices ($\chi^{(0)}$,$\chi$) of each atom are collected to post-process them and compute the final U/V values using $V_{IJ} = (\chi^{(0) -1} -\chi^{-1})_{IJ}$

As for the `HpBaseWorkChain`, we also have here the `hubbard_structure` output namespace, containing the same results as the serial execution:

In [ ]:
from aiida_quantumespresso.utils.hubbard import HubbardUtils
print(HubbardUtils(results['hubbard_structure']).get_hubbard_card())

## Parallelize q points for each perturbed atom

In density-functional perturbation theory, we can simulate linear responses in reciprocal space as monocrhomatic perturbations, described via a grid of __q points__: each q point a monocrhomatic perturbation. The number of q points can be reduced using symmetries, and each Hubbard atom (manifold) will have in principle different number of perturbations.

Specifying the input `parallelize_qpoints` as `True` in `HpWorkChain`, each single independent q point _of each atom_ will run as a separate `HpBaseWorkChain`.

:::{important}
To parallelize over q points you __MUST__ parallelize over atoms as well.
:::

In [ ]:
builder = HpWorkChain.get_builder_from_protocol(
    code=data.hp_code,
    protocol="fast",
    parent_scf_folder=pw_node.outputs.remote_folder,
    overrides={
        "parallelize_atoms":True, 
        "parallelize_qpoints":True,  
        "hp":{"hubbard_structure":data.structure},
        "qpoints_distance": 1000, # to get few q points
        "max_concurrent_base_workchains": 2, # useful to not overload HPC or local computer
    }
)

results, hp_node = run_get_node(builder)

In [ ]:
%verdi process status {hp_node.pk}

The following just happened:
- A grid of q points was generated automatically using the distance (between points) in $\r{A}^{-1}$ we gave in input (of 1000 $\r{A}^{-1}$ to have very sparse - it is just a tutorial!).
- The `HpParallelizeAtomsWorkChain` is called.
- This work chain calls first a `HpBaseWorkChain` to get the independent atoms to perturb.
- For independent each atom (three in total) an `HpParallelizeQpointsWorkChain` is submitted __simultaneously__, one for cobalt, and two for the two oxygen sites.
- Each of such work chain submit a fist `HpBaseWorkChain` to get the independent q points (in this case, only 1).
- An `HpBaseWorkCahin` is run for every q points, executed at the same time. __Imagine this on an HPC!__ :rocket:
- The response matrices ($\chi^{(0)}_{\mathbf{q}}$,$\chi_{\mathbf{q}}$) of each q point for each atom are collected to post-process them and compute the atomic response matrices.
- A last final `HpBaseWorkChain` collects such matrices to compute U/V values.

And we check the results are the same as before:

In [ ]:
print(HubbardUtils(results['hubbard_structure']).get_hubbard_card())

## Final considerations

We managed to compute the Hubbard parameters __parallelizing over atoms and q points__! :tada:

Still, you might need to converge self-consistently the parameters using the iterative procedure of relax -> scf -> hubbard.
Learn the automated way [in the last tutorial](./3_self_consistent.ipynb)!

:::{admonition} Learn more and in details
:class: hint

To learn the full sets of inputs, to use proficiently the `get_builder_from_protocol` and more, have a look at the following sections:
- [Specific how tos](howto/workflows/hp/main.md)
- [General information of the implemented workchain](topics/workflows/hp/main.md)
:::